In [1]:
%pylab inline
from nltk.corpus import brown

Populating the interactive namespace from numpy and matplotlib


In [2]:
docs = []
word_to_count = {}
for fileid in brown.fileids():
    doc = []
    for word in brown.words(fileids=fileid):
        word = word.lower()
        if word not in word_to_count:
            word_to_count[word] = 0
        word_to_count[word] += 1
        doc.append(word)
    docs.append(doc)

In [3]:
word_to_id = {w: i for i, (w, c) in enumerate(sorted(word_to_count.items(), key=lambda x: -x[1]))}
docs_ids = []
for doc in docs:
    doc_ids = []
    for word in doc:
        doc_ids.append(word_to_id[word])
    docs_ids.append(doc_ids)

In [9]:
doc_train = docs_ids[:450]
doc_eval = docs_ids[450:]
n = 4

ngram_to_count = {}
for doc in doc_train:
    for i in xrange(len(doc)):
        ngram = tuple(doc[i:i + n])
        if len(ngram) < n:
            break
        if ngram not in ngram_to_count:
            ngram_to_count[ngram] = 0
        ngram_to_count[ngram] += 1

In [10]:
ngram_posterior = {}
for ngram, count in ngram_to_count.items():
    context = ngram[:n - 1]
    target = ngram[n - 1]
    if context not in ngram_posterior:
        ngram_posterior[context] = {}
    if target not in ngram_posterior[context]:
        ngram_posterior[context][target] = 0
    ngram_posterior[context][target] += count

In [11]:
eps = 1e-6
log_prob_cum = 0.0
nsamples = 0
for doc in doc_eval:
    for i in xrange(len(doc)):
        ngram = tuple(doc[i:i + n])
        if len(ngram) < n:
            break
        context = ngram[:n - 1]
        target = ngram[n - 1]
        if context in ngram_posterior:
            distn = ngram_posterior[context]
            total_samples = float(sum(distn.values()))
            if target in distn:
                p = distn[target] / total_samples
            else:
                p = eps
        else:
            p = eps
        log_prob_cum += np.log2(p)
        nsamples += 1
negative_log_prob = -log_prob_cum / float(nsamples)
print negative_log_prob
print np.exp2(negative_log_prob)

18.5403201473
381234.761692
